# Model Experimentation: Hyperparameter Optimization
This notebook conducts experiments on hyperparameter optimization.

## Imports and Setup

In [1]:
import pandas as pd
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Location

In [2]:
version = '20200502'
data_dir = "../data/"
input_file = data_dir + '{}_dataset.csv'.format(version)

images_dir = data_dir + 'images/'
indices_dir = data_dir + 'indices/'
pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'

## Load Dataset

In [3]:
%%time
data = pd.read_csv(input_file).reset_index(drop=True)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (1241343, 113)
CPU times: user 31.4 s, sys: 1.9 s, total: 33.3 s
Wall time: 32.3 s


,B1_2015-2016,B2_2015-2016,B3_2015-2016,B4_2015-2016,B5_2015-2016,B6_2015-2016,B7_2015-2016,B8_2015-2016,B9_2015-2016,B10_2015-2016,...,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,la_guajira,target,area
0,0.15100,0.13225,0.14240,0.16430,0.1899,0.25700,0.2960,0.25510,0.32295,0.0396,...,-0.509745,0.105128,0.239614,0.449106,-0.718433,-0.042537,0.901237,1,3,0
1,0.15100,0.12885,0.13790,0.16185,0.1899,0.25700,0.2960,0.25755,0.32295,0.0396,...,-0.507485,0.105128,0.247826,0.464498,-0.716955,-0.039760,0.911490,1,3,0
2,0.15895,0.13730,0.15185,0.18915,0.2264,0.28555,0.3268,0.28085,0.35740,0.0416,...,-0.524371,0.073259,0.262348,0.446475,-0.722188,-0.033995,0.875915,1,3,0


## Resample Dataset

In [4]:
data = model_utils.resample(data, num_neg_samples=30000, random_state=SEED)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (297404, 113)


,B1_2015-2016,B2_2015-2016,B3_2015-2016,B4_2015-2016,B5_2015-2016,B6_2015-2016,B7_2015-2016,B8_2015-2016,B9_2015-2016,B10_2015-2016,...,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,la_guajira,target,area
0,0.1681,0.1326,0.1329,0.1304,0.1571,0.2687,0.3205,0.2784,0.3581,0.0370,...,-0.495702,0.213802,0.122766,0.386654,-0.762164,-0.097429,1.005492,1,2,0
1,0.1933,0.1974,0.1994,0.2195,0.2191,0.2680,0.3012,0.2700,0.3280,0.0384,...,-0.417789,0.102637,0.211351,0.520237,-0.748431,-0.021367,1.030898,1,2,0
2,0.1946,0.1782,0.1828,0.1908,0.2174,0.2747,0.3062,0.2910,0.3234,0.0307,...,-0.188068,0.079984,0.222447,0.746002,-0.641025,-0.030201,1.056083,1,2,0


## Machine Learning Pipeline

In [5]:
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
SEED = 42

## Define Features and Target Label

In [6]:
label = 'target'
features = [column  for column in data.columns[:-2]]
data[label] = data[label].replace({2:0, 3:0})

splits = data[['area']]
X = data[features]
y = data[label]

print('Data dimensions: {}'.format(data.shape))
print('Class distribution:\n{}'.format(data['target'].value_counts()))

Data dimensions: (297404, 113)
Class distribution:
0    270000
1     27404
Name: target, dtype: int64


## Ridge Classifier

In [7]:
%%time
rc =  RidgeClassifier()
param_grid = {'classifier__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
rc_results = model_utils.nested_spatial_cv(rc, X, y, splits=splits, param_grid=param_grid, verbose=2);

Test Set: Maicao
Predicted  False  True  __all__
Actual                         
False      28251  1749    30000
True         143   567      710
__all__    28394  2316    30710

               precision    recall  f1-score   support

           0       0.99      0.94      0.97     30000
           1       0.24      0.80      0.37       710

    accuracy                           0.94     30710
   macro avg       0.62      0.87      0.67     30710
weighted avg       0.98      0.94      0.95     30710

F1 Score: 0.3748
Kappa Statistics: 0.3518
Precision: 0.2448
Recall: 0.7986
Accuracy: 0.9384
ROC AUC: 0.8701

Test Set: Riohacha
Predicted  False  True  __all__
Actual                         
False      29984    16    30000
True        1978  1523     3501
__all__    31962  1539    33501

               precision    recall  f1-score   support

           0       0.94      1.00      0.97     30000
           1       0.99      0.44      0.60      3501

    accuracy                           0

## Naive Bayes

In [8]:
%%time
gnb = GaussianNB()
gnb_results = model_utils.nested_spatial_cv(
    gnb, X, y, splits=splits, param_grid=None, search_type=None, feature_selection=None, verbose=2
);

Test Set: Maicao
Predicted  False   True  __all__
Actual                          
False      15924  14076    30000
True           0    710      710
__all__    15924  14786    30710

               precision    recall  f1-score   support

           0       1.00      0.53      0.69     30000
           1       0.05      1.00      0.09       710

    accuracy                           0.54     30710
   macro avg       0.52      0.77      0.39     30710
weighted avg       0.98      0.54      0.68     30710

F1 Score: 0.0916
Kappa Statistics: 0.0497
Precision: 0.0480
Recall: 1.0000
Accuracy: 0.5416
ROC AUC: 0.7654

Test Set: Riohacha
Predicted  False   True  __all__
Actual                          
False      20252   9748    30000
True         573   2928     3501
__all__    20825  12676    33501

               precision    recall  f1-score   support

           0       0.97      0.68      0.80     30000
           1       0.23      0.84      0.36      3501

    accuracy                  

## Logistic Regression

In [9]:
%%time
lr =  LogisticRegression()
param_grid = {'classifier__C':[0.001, 0.01, 1.0, 5.0, 10]}
lr_results = model_utils.nested_spatial_cv(lr, X, y, splits=splits, param_grid=param_grid, verbose=2);

Test Set: Maicao
Predicted  False  True  __all__
Actual                         
False      27895  2105    30000
True         316   394      710
__all__    28211  2499    30710

               precision    recall  f1-score   support

           0       0.99      0.93      0.96     30000
           1       0.16      0.55      0.25       710

    accuracy                           0.92     30710
   macro avg       0.57      0.74      0.60     30710
weighted avg       0.97      0.92      0.94     30710

F1 Score: 0.2456
Kappa Statistics: 0.2174
Precision: 0.1577
Recall: 0.5549
Accuracy: 0.9212
ROC AUC: 0.7424

Test Set: Riohacha
Predicted  False  True  __all__
Actual                         
False      29986    14    30000
True        2081  1420     3501
__all__    32067  1434    33501

               precision    recall  f1-score   support

           0       0.94      1.00      0.97     30000
           1       0.99      0.41      0.58      3501

    accuracy                           0

## LinearSVC

In [10]:
%%time
lsvc = LinearSVC(random_state=SEED)
param_grid = {'classifier__C': [0.001, 0.1, 0.5, 1, 5, 10, 100, 1000]}
lsvc_results = model_utils.nested_spatial_cv(lsvc, X, y, splits=splits, param_grid=param_grid, verbose=2);

Test Set: Maicao
Predicted  False  True  __all__
Actual                         
False      27790  2210    30000
True         300   410      710
__all__    28090  2620    30710

               precision    recall  f1-score   support

           0       0.99      0.93      0.96     30000
           1       0.16      0.58      0.25       710

    accuracy                           0.92     30710
   macro avg       0.57      0.75      0.60     30710
weighted avg       0.97      0.92      0.94     30710

F1 Score: 0.2462
Kappa Statistics: 0.2178
Precision: 0.1565
Recall: 0.5775
Accuracy: 0.9183
ROC AUC: 0.7519

Test Set: Riohacha
Predicted  False  True  __all__
Actual                         
False      28839  1161    30000
True         348  3153     3501
__all__    29187  4314    33501

               precision    recall  f1-score   support

           0       0.99      0.96      0.97     30000
           1       0.73      0.90      0.81      3501

    accuracy                           0

## Random Forest

In [ ]:
%%time
param_grid = {
    'classifier__n_estimators': [int(x) for x in np.linspace(start = 10, stop = 500, num = 10)],
    'classifier__max_features': ['auto', 'sqrt'],
    'classifier__max_depth': [5, 6, 7, 8, 9, 10],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__bootstrap': [True, False]
}
rf = RandomForestClassifier(random_state=SEED)
model_utils.nested_spatial_cv(rf, X, y, splits=splits, param_grid=param_grid, search_type='random', verbose=2);